In [8]:
import json
import numpy as np
from glob import glob

In [13]:
data = []
for filename in glob('../results/2D*.json'):
    with open(filename, 'r') as f:
        data += json.load(f)

In [14]:
faulty = [o for o in data if len(o['times'])==0]
for o in faulty:
    print(o['command'])

OMP_NUM_THREADS=8 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/ijk/out/TMM_parallel_I 2500 8 8 8
OMP_NUM_THREADS=12 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/ijk/out/TMM_parallel_J 2500 8 8 8
/s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Sequential/tiled/ijk/out/TMM 2500 16 16 16
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/ijk/out/TMM_parallel_I 2500 16 16 16
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/ijk/out/TMM_parallel_J 2500 16 16 16
OMP_NUM_THREADS=6 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/ijk/out/TMM_parallel_J 2500 16 16 16
OMP_NUM_THREADS=10 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/ijk/out/TMM_parallel_J 2500 16 16 16
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScript

OMP_NUM_THREADS=4 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_I 5000 48 48 48
OMP_NUM_THREADS=6 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_J 5000 48 48 48
OMP_NUM_THREADS=8 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_I 5000 48 48 48
OMP_NUM_THREADS=8 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_J 5000 48 48 48
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_I 5000 64 64 64
OMP_NUM_THREADS=4 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_I 5000 64 64 64
OMP_NUM_THREADS=6 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kij/out/TMM_parallel_I 5000 64 64 64
OMP_NUM_THREADS=8 /s/chopin/l/grad/lnarmour/git/

OMP_NUM_THREADS=6 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 4000 400 400 400
OMP_NUM_THREADS=10 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_I 4000 400 400 400
OMP_NUM_THREADS=12 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 4000 400 400 400
/s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Sequential/tiled/kji/out/TMM 4000 416 416 416
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_I 4000 416 416 416
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 4000 416 416 416
OMP_NUM_THREADS=4 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_I 4000 416 416 416
OMP_NUM_THREADS=8 /s/chopin/l/grad/lnarmour/git/parr

OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_I 1000 112 112 112
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_J 1000 144 144 144
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_J 1000 176 176 176
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_J 1000 208 208 208
OMP_NUM_THREADS=8 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_I 1000 240 240 240
OMP_NUM_THREADS=10 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_I 1000 272 272 272
OMP_NUM_THREADS=10 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/jik/out/TMM_parallel_I 1000 304 304 304
OMP_NUM_THREADS=8 /s/chop

OMP_NUM_THREADS=10 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 5500 304 304 304
OMP_NUM_THREADS=12 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_I 5500 304 304 304
OMP_NUM_THREADS=12 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 5500 304 304 304
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_I 5500 320 320 320
OMP_NUM_THREADS=2 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 5500 320 320 320
OMP_NUM_THREADS=4 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 5500 320 320 320
OMP_NUM_THREADS=6 /s/chopin/l/grad/lnarmour/git/parric-ttmm/scripts/finalScripts/2D-Parallel/tiled/kji/out/TMM_parallel_J 5500 320 320 320
OMP_NUM_THREADS=8 /s/cho

In [10]:
data = [o for o in data if len(o['times'])!=0]
        
X, Y = [], []
for x in data:
    perm_map = {'ijk':1, 'ikj':2, 'jik':3, 'jki':4, 'kij':5, 'kji':6}
    N = x['N']
    permutation = perm_map[x['permutation']]
    tiled = 1 if x['tiled'] else 0
    TS = x['TS']
    parallel = 1 if x['parallel'] else 0
    loop_parallelized = 0 if x['loop_parallelized']=='I' else 0
    num_threads = x['num_threads']
    time = np.mean(sorted(x['times'])[1:-1])
    
    X.append([N, permutation, tiled, TS, parallel, loop_parallelized, num_threads])
    Y.append(time)

#G.append((N ** 3) / 3 / time / (10 ** 9))
X = np.array(X)
Y = np.array(Y)
Y.shape = (Y.shape[0], 1)
P = np.hstack([X,Y])

In [11]:
P

array([[2500, 1, 1, ..., 0, 10, 1.5355082000000002],
       [2500, 1, 1, ..., 0, 12, 1.5928032],
       [2500, 1, 1, ..., 0, 8, 1.6551048000000002],
       ...,
       [3000, 6, 1, ..., 0, None, 45.4892068],
       [3000, 6, 1, ..., 0, 6, 44.4174962],
       [3000, 6, 1, ..., 0, 8, 45.1753468]], dtype=object)